# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

Нам предоставили данные по тарифам мобильной связи. Для начала анализа, мы должны импортировать все библиотеки для дальнейшей работы. Хоть предобработка данных нам не требуется, но мы все равно проверим их на наличие ошибок в типах данных, дубликатов и пропусков.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier


In [2]:
data=pd.read_csv('/datasets/users_behavior.csv')

data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [3]:
# Посмотрим общую информацию
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Итак, мы открыли файл и просмотрели его. Пропусков нет, типы данных все верные. 

In [4]:
# Проверим явные дубликаты
data.duplicated().sum()

0

Дубликатов нет, пропусков нет, данные чистые. Рассмотрим их.

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц.

сalls — количество звонков,

minutes — суммарная длительность звонков в минутах,

messages — количество sms-сообщений,

mb_used — израсходованный интернет-трафик в Мб,

is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Теперь можно приступить к анализу.

## Разбейте данные на выборки

Разобьем данные на 3 выборки: обучающая, тестовая и валидационная. Так как тестовая выборка не спрятана, то разбиваем на 3 выборки в соотношении 60%:20%:20%.

In [5]:
# Разобьем данные на признаки и целевой признак, по которым будем обучать
features = data.drop(['is_ultra'], axis=1) 
target = data['is_ultra']

# Теперь разделим данные на обучающую выборку(60%) и тестовую (отбираем 40%)
features_train, features_test, target_train, target_test=train_test_split(features,
                                                                          target,
                                                                          test_size=0.4,
                                                                          random_state=12345)

# Сейчас мы тестовую выборку делим пополам на тестовую(20%) и валидационную(20%)
features_test, features_valid, target_test, target_valid=train_test_split(features_test,
                                                                          target_test,
                                                                          test_size=0.5,
                                                                          random_state=12345)

# Проверим размер выборок
print('Размер обучающей выборки:', features_train.shape)
print('Размер тестовой выборки:', features_test.shape)
print('Размер валидационной выборки:', features_valid.shape)

Размер обучающей выборки: (1928, 4)
Размер тестовой выборки: (643, 4)
Размер валидационной выборки: (643, 4)


**Вывод**: Мы разделили наши данные на 3 выборки. Обучающая, тестовая и валидационная. По стандарту, валидационной выборке выделяют 25% от всех данных, но это при условии, что тестовая выборка уже имеется. Так как у нас не такая ситуация, мы делим данные в соотношении 3:1:1. По итогу нашего выделения, выборки соответствуют данной пропорции.

## Исследуйте модели

Есть множество моделей для обучения, мы используем Дерево решений, Случайный лес и Логистическую регрессию. Выберем самую удачную модель, а именно проверим значение accuracy- оно не должно быть менее 0.75. Для этого будем менять гиперпараметры.

In [6]:
# Проверим модель Дерево решений, меняя гиперпараметр max_depth
best_model_dtc=None
best_result_dtc=0
best_depth=0
for depth in range(1,10):
    model_dtc=DecisionTreeClassifier(random_state=12345, max_depth=depth).fit(features_train,
                                                                              target_train)
    predictions_valid_dtc=model_dtc.predict(features_valid)
    result_dtc=accuracy_score(target_valid, predictions_valid_dtc)
    
    if result_dtc>best_result_dtc:
        best_model_dtc=model_dtc
        best_result_dtc=result_dtc
        best_depth=depth
        
print('Accuracy модели Дерево решений:', best_result_dtc, 'Лучшая глубина дерева:',best_depth, end='')
print()

# Проверим модель Случайный лес, меняя гиперпараметр n_estimators
best_model_rfc=None
best_result_rfc=0
best_est=0
for est in range(1,10):
    model_rfc=RandomForestClassifier(random_state=12345, n_estimators=est).fit(features_train, 
                                                                               target_train)
    predictions_valid_rfc=model_rfc.predict(features_valid)
    result_rfc=accuracy_score(target_valid, predictions_valid_rfc)
    
    if result_rfc>best_result_rfc:
        best_model_rfc=model_rfc
        best_result_rfc=result_rfc
        best_est=est
        
print('Accuracy модели Случайный лес:', best_result_rfc, 'Лучшее кол-во деревьев:',best_est, end='')
print()


# Проверим модель Логистическая регрессия, меняя гиперпараметр  max_iter
iters=100
model_lg=LogisticRegression(random_state=12345, solver='lbfgs', max_iter=iters)
model_lg.fit(features_train, target_train)
predictions_valid_lg=model_lg.predict(features_valid)
result_lg=accuracy_score(predictions_valid_lg, target_valid)

print('Accuracy модели Логистическая регрессия:',result_lg, 'Лучшее кол-во итераций:',iters, end='')       

Accuracy модели Дерево решений: 0.7993779160186625 Лучшая глубина дерева: 7
Accuracy модели Случайный лес: 0.7869362363919129 Лучшее кол-во деревьев: 8
Accuracy модели Логистическая регрессия: 0.6842923794712286 Лучшее кол-во итераций: 100

**Вывод**: Мы рассмотрели 3 модели обучения. Дерево решений,Случайный лес и Логистическая регрессия. Для каждой модели мы определили accuracy, и лучшие гиперпараметры. По результатам анализа, доля правильных ответов больше всего у модели Дерево решений, а у Логистической регрессии показатель не дотягивает даже до минимума 0.75( результат не менялся даже при разных гиперпараметрах)

## Проверьте модель на тестовой выборке

Так как лучшие показатели были у модели Дерево решений, то проверим тестовую выборку именно на ней

In [7]:
model_dtc=DecisionTreeClassifier(random_state=12345, max_depth=7).fit(features_train,
                                                                              target_train)
predictions_test_dtc=model_dtc.predict(features_test)
result_test_dtc=accuracy_score(target_test, predictions_test_dtc)

print('Accuracy модели Дерево решений на тестовой выборке:', result_test_dtc)

Accuracy модели Дерево решений на тестовой выборке: 0.7822706065318819


**Вывод**: Доля правильных ответов на тестовой выборке меньше, чем на валидационной. 

## (бонус) Проверьте модели на адекватность

Для проверки адекватности модели используют DummyClassifier

In [10]:
model_dc = DummyClassifier(strategy='most_frequent', random_state=12345)
model_dc.fit(features_train, target_train)
result_dc = model_dc.score(features_test, target_test)
print('Accuracy модели DummyClassifier:', result_dc)

Accuracy модели DummyClassifier: 0.7060653188180405


**Вывод:** Accuracy модели DummyClassifier: 0.71. Accuracy модели Дерево решений на тестовой выборке: 0.78. Наша модель победила по метрике качества "неадекватную" модель, так что проверку мы прошли.

## Общий вывод.
При проверки предоставленных данных не было обнаружено никаких критических ошибок и пропусков. Дубликаты не обнаружены, типы данных все соответствуют. Данные чистые и обрабоданные.

Данные разделили на 3 выборки: обучающую, тестовую и валидационную. Соотношение выборок взяли 3:1:1, так как тестовая выборка не была "спрятана".
Мы провели исследование 3 моделей на основе наших выборок: Decision Tree Classifier, Random Forest Classifier и Logistic Regression; изучили их гиперпараметры и определили лучшие показатели. Результатами модели Decision Tree Classifier мы остались довольны, accuracy была выше объявленного нами минимума 0.75( accuracy = 0.799). Так же модель прошла тест на адекватность при сравнении ее с моделью DummyClassifier(accuracy Decision Tree Classifier 0.79 > accuracy DummyClassifier 0.71)

Модель Logistic Regression в данном анализе показала себя хуже всех, скорость загрузки и accuracy были ниже, чем у других моделей. Возможно, при других выборках, эта модель себя проявит лучше.
